In [ ]:
#!pip install numpy==1.15.4
#!pip install deepdish
#!pip install tensorflow==1.15

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import deepdish as dd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ISIS_dataset')

In [ ]:
#!pip install numpy==1.15.4

In [ ]:
data = dd.io.load('ISIC_dataset.h5')
data_df = pd.DataFrame(data)

In [ ]:
X_raw = data_df.image
Y = data_df.cell_type_idx

In [ ]:
Y.iloc[:3]

In [ ]:
#Asi se divide los labels
prueba = Y[Y == 2]

In [ ]:
prueba.value_counts()

In [ ]:
Y.value_counts()

In [ ]:
Y.unique()

In [ ]:
Y.iloc[1]

In [ ]:
New_X = []
for i in range(len(Y)):
  if Y.iloc[i] == 4:
    New_X.append(X_raw.iloc[i])
print(len(New_X))
New_X_Array = np.array(New_X).reshape(-1,75,100,3)
New_X_Array.shape
plt.imshow(New_X_Array[1,:,:,:])

In [ ]:
New_X_Array = np.array(New_X).reshape(-1,75,100,3)

In [ ]:
print(New_X_Array.shape)
plt.imshow(New_X_Array[1,:,:,:])

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
First_Channel = New_X_Array[:2000,:,:,0]
Second_Channel = New_X_Array[:2000,:,:,1]
Third_Channel = New_X_Array[:2000,:,:,2]

In [ ]:
print(First_Channel.shape)
print(Second_Channel.shape)
print(Third_Channel.shape)

In [ ]:
First_Channel = First_Channel.reshape(-1,7500)
Second_Channel = Second_Channel.reshape(-1,7500)
Third_Channel = Third_Channel.reshape(-1,7500)

In [ ]:
print(First_Channel.shape)
print(Second_Channel.shape)
print(Third_Channel.shape)

In [ ]:
scaler.fit(First_Channel)
scaler.fit(Second_Channel)
scaler.fit(Third_Channel)
First_Channel_sc = scaler.transform(First_Channel)
Second_Channel_sc= scaler.transform(Second_Channel)
Third_Channel_sc= scaler.transform(Third_Channel)

In [ ]:
DATOS = [First_Channel_sc,Second_Channel_sc,Third_Channel_sc]

In [ ]:
def PCA_RGB (DATO):
  pca = PCA(n_components=2000)
  j=0
  for i in DATO:
    j+=1
    pca.fit(i)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    print("channel",j)
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance')
    plt.show()

In [ ]:
PCA_RGB(DATOS)

In [ ]:

def NEW_PCA_ARRAY (DATOS,num_comp):
  DATOS_PCA =[]
  STD_PCA = []
  j = 0 
  for i in DATOS: 
    pca = PCA(n_components=num_comp[j])
    X_pca_array = pca.fit_transform(i)
    DATOS_PCA.append(X_pca_array)
    pca_std = np.std(X_pca_array)
    STD_PCA.append(pca_std)
    j+=1
  return (DATOS_PCA,STD_PCA)

In [ ]:
NUM_COMP=[750,750,750]

In [ ]:
ChannelNewComponents = NEW_PCA_ARRAY(DATOS,NUM_COMP)

In [ ]:
NEW_CHANNELS = ChannelNewComponents[0]

In [ ]:
pca = PCA(n_components=6700)
for i in DATO:
   pca.fit(i)

In [ ]:
def INVERSA_PCA_SC_to_IMAGE(datosPCA,DATO):
  pca = PCA(n_components=750)
  for i in DATO:
   pca.fit(i)
  INV_PCA = []
  for i in datosPCA:
    inv_pca = pca.inverse_transform(i)
    inv_sc = scaler.inverse_transform(inv_pca)
    INV_PCA.append(inv_sc)
  return INV_PCA

In [ ]:
NEW_CHANNELS_INV = INVERSA_PCA_SC_to_IMAGE(NEW_CHANNELS,DATOS)

In [ ]:
print(NEW_CHANNELS_INV[0].shape)
print(NEW_CHANNELS_INV[1].shape)
print(NEW_CHANNELS_INV[2].shape)

In [ ]:
import cv2

In [ ]:
images = cv2.merge((NEW_CHANNELS_INV[0],NEW_CHANNELS_INV[1],NEW_CHANNELS_INV[2]))

In [ ]:
print(images.shape)
images = images.reshape(-1,75,100,3)
print(images.shape)

In [ ]:
New_X_Array

In [ ]:
plt.imshow(New_X_Array[0])

In [ ]:
images = images/images.max()

In [ ]:
plt.imshow(images[0])

In [ ]:
def side_by_side(indexes):
    org = New_X_Array[indexes]/New_X_Array.max()
    rec = images[indexes]
    pair = np.concatenate((org, rec), axis=1)
    plt.figure(figsize=(8,6))
    plt.imshow(pair)
    plt.show()
    
for index in range(5):
    side_by_side(index)